In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from decimal import Decimal
from typing import List, Tuple
from warnings import warn

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml
from ogb.nodeproppred import PygNodePropPredDataset

import tqdm
tqdm.tqdm.pandas()

In [4]:
from notebooks import mpl_latex

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale_refactoring/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [5]:
mpl_latex.enable_production_mode()

In [6]:
from rgnn_at_scale.data import prep_graph, split
from rgnn_at_scale.attacks import create_attack, SPARSE_ATTACKS
from rgnn_at_scale.helper.io import Storage
from rgnn_at_scale.models import DenseGCN, GCN
from rgnn_at_scale.train import train
from rgnn_at_scale.helper.utils import accuracy

# Directed graph for disentangeling intent

**General config:**

In [7]:
datasets = ['cora_ml', 'citeseer', 'pubmed', 'ogbn-arxiv']
search_space_sizes = [1_000_000, 2_500_000, 10_000_000]
binary_attr = False
attack = 'PRBCD'
seed = 0
device = 0
victim_params = {
    'n_filters': [256, 256],
    'dropout': 0.5,
    'with_batchnorm': True,
}
train_params = {
    'lr': 1e-2,
    'weight_decay': 0,
    'patience': 300,
    'max_epochs': 3000
}
attack_params = {
    'keep_heuristic': 'WeightOnly',
    'loss_type': 'tanhMargin',
    'fine_tune_epochs': 0,
    'with_early_stropping': False
}
epsilons = [0.001, 0.01, 0.1]
display_steps = 10

y_kde_max = 7.5

make_undirected_options = [True, False]
loss_types = ['CE', 'CW', 'tanhMargin']

torch.manual_seed(seed)
np.random.seed(seed)

**Config for plotting:**

In [8]:
bins = 6

assert bins % 2 == 0, 'Only an even number of bins supported'

x_0 = 4
x_1 = 2
x_2 = 0

bar_width = 0.8

neg_colormap = plt.get_cmap('Oranges')
pos_colormap = plt.get_cmap('Blues')

colors = (
    [neg_colormap((bins / 2 - i) / bins) for i in range(bins // 2)]
    + [pos_colormap((bins / 2 - i) / bins) for i in reversed(range(bins // 2))]
)

fig_width=0.5
fig_ratio_yx=0.4

**Config for attacks:**

In [9]:
for dataset, search_space_size in zip(datasets, search_space_sizes):
    for make_undirected in [False, True]:

        graph = prep_graph(dataset, device='cpu', binary_attr=binary_attr, make_undirected=make_undirected,
                           return_original_split=dataset.startswith('ogbn'))
        
        attr, adj, labels = graph[:3]
        if len(graph) == 3:
            idx_train, idx_val, idx_test = split(labels.cpu().numpy())
        else:
            idx_train, idx_val, idx_test = graph[3]['train'], graph[3]['valid'], graph[3]['test']
            
        n_features = attr.shape[1]
        n_classes = int(labels.max() + 1)

        victim_params_current = dict(victim_params)
        if make_undirected == False:
            victim_params['row_norm'] = True
            victim_params_current['n_filters'] = []

        gcn = GCN(n_classes=n_classes, n_features=n_features, **victim_params_current).to(device)
        train(model=gcn, attr=attr.to(device), adj=adj.to(device), labels=labels.to(device),
              idx_train=idx_train, idx_val=idx_val, display_step=display_steps, **train_params)

        gcn.eval()

        with torch.no_grad():
            clean_logits = gcn(attr.to(device), adj.to(device))
        accuracy(clean_logits, labels.to(device), idx_test)
        print(f'Clean accuracy: {accuracy(clean_logits, labels.to(device), idx_test)}')

        for epsilon in epsilons:
            for loss_type in loss_types:
                print(dataset, make_undirected, epsilon, loss_type)
                
                temp_attack_params = dict(attack_params)
                temp_attack_params['loss_type'] = loss_type
                temp_attack_params['epsilon'] = epsilon
                temp_attack_params['search_space_size'] = search_space_size
                
                adversary = create_attack(attack, adj=adj, attr=attr, binary_attr=False, labels=labels, 
                                          make_undirected=make_undirected, model=gcn, idx_attack=idx_test, 
                                          device=device, data_device=device, **temp_attack_params)

                m = adj.nnz()
                torch.manual_seed(seed)
                np.random.seed(seed)

                n_perturbations = int(round(epsilon * m))
                adversary.attack(n_perturbations)
                
                ############## Attack Loss ###############
                
                fig, ax = mpl_latex.newfig(width=fig_width, ratio_yx=fig_ratio_yx)

                color = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
                ax.set_xlabel('Epoch $t$')
                ax.set_ylabel(r"Surrogate loss $\mathcal{L}'$", color=color)
                ax.plot(adversary.attack_statistics['loss'], color=color)
                ax.tick_params(axis='y', labelcolor=color)

                ax = ax.twinx()

                color = plt.rcParams['axes.prop_cycle'].by_key()['color'][1]
                ax.set_ylabel('Accuracy', color=color)
                ax.plot(adversary.attack_statistics['accuracy'], color=color)
                ax.tick_params(axis='y', labelcolor=color)
                
                plt.gcf().show()

                mpl_latex.savefig(
                    f'latex/assets/global_prbcd_attack_loss_{dataset}_{make_undirected}_{epsilon}_{loss_type}',
                    fig, close_fig=True
                )
                
                ############################################

                if make_undirected:
                    attcked_edges = adversary.modified_edge_index[
                        :, adversary.modified_edge_weight_diff.round().bool()
                    ].flatten()
                else:
                    attcked_edges = adversary.modified_edge_index[
                        0, adversary.modified_edge_weight_diff.round().bool()
                    ].flatten()
                attcked_edges = attcked_edges[np.isin(attcked_edges.cpu().numpy(), idx_test)]

                with torch.no_grad():
                    attacked_logits = gcn(attr.to(device), adversary.adj_adversary.to(device))
                print(f'Perturbed accuracy: {accuracy(attacked_logits, labels.to(device), idx_test)}')

                sorted_attacked = attacked_logits.argsort(-1).cpu()
                best_non_target_class_attacked = sorted_attacked[sorted_attacked != labels[:, None]].reshape(attacked_logits.size(0), -1)[:, -1]
                margin_attacked = (
                    F.softmax(attacked_logits, dim=-1)[np.arange(attacked_logits.size(0)), labels.long()]
                    - F.softmax(attacked_logits, dim=-1)[np.arange(attacked_logits.size(0)), best_non_target_class_attacked.long()]
                )

                sorted_clean = clean_logits.argsort(-1).cpu()
                best_non_target_class = sorted_clean[sorted_clean != labels[:, None]].reshape(clean_logits.size(0), -1)[:, -1]
                margin = (
                    F.softmax(clean_logits, dim=-1)[np.arange(clean_logits.size(0)), labels.long()]
                    - F.softmax(clean_logits, dim=-1)[np.arange(clean_logits.size(0)), best_non_target_class.long()]
                )

                freq_all_clean, borders = np.histogram(margin.cpu(), bins=bins, range=(-1,1), density=True)
                freq_attacked_clean, _ = np.histogram(margin[attcked_edges].cpu(), bins=bins, range=(-1,1), density=True)
                freq_attacked_pert, _ = np.histogram(margin_attacked[attcked_edges].cpu(), bins=bins, range=(-1,1), density=True)

                freq_all_clean = 2 / bins * np.cumsum(freq_all_clean)
                freq_attacked_clean = 2 / bins * np.cumsum(freq_attacked_clean)
                freq_attacked_pert = 2 / bins * np.cumsum(freq_attacked_pert)

                ############## Bar Plot ###############
                
                fig, ax = mpl_latex.newfig(width=fig_width, ratio_yx=fig_ratio_yx)

                for i, (el_0, el_1, el_2, color) in enumerate(zip(freq_all_clean, freq_attacked_clean, freq_attacked_pert, colors)):
                    if i != 0:
                        sep = '<'
                    else:
                        sep = '\le'
                    plt.barh([x_0, x_1, x_2],
                             (el_0, el_1, el_2), 
                             bar_width,
                             color=color,
                             zorder=bins - i,
                             label=f'${borders[i]:.1f} {sep} \psi \le {borders[i + 1]:.2f}$')
                    if i < bins-1:
                        plt.plot((el_0, el_1), [x_0 - bar_width / 2, x_1 + bar_width / 2], color='grey', label='_nolegend_')
                        plt.plot((el_1, el_2), [x_1 - bar_width / 2, x_2 + bar_width / 2], color='grey', label='_nolegend_')

                plt.yticks([x_0, x_1, x_2], ['All nodes\n(clean)', 'Att. nodes\n(clean)', 'Att. nodes\n(pert.)'])

                plt.grid(False)

                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_visible(False)
                ax.spines['top'].set_visible(False)

                plt.xlabel('Frequency')
                plt.legend(bbox_to_anchor=(1.05, 1), title=r'Class. margin', loc='upper left')

                plt.gcf().show()

                mpl_latex.savefig(
                    f'latex/assets/global_prbcd_bar_{dataset}_{make_undirected}_{epsilon}_{loss_type}_relfeq_margin_attacked',
                    fig, close_fig=True
                )
                
                ############################################
                
                ############## KDE Plot ###############
                
                fig, ax = mpl_latex.newfig(width=fig_width, ratio_yx=fig_ratio_yx)

                kde1 = sns.kdeplot(margin.cpu().numpy(), ax=ax, label='All nodes (clean)')
                kde2 = sns.kdeplot(margin[attcked_edges].cpu().numpy(), ax=ax, label='Att. nodes (clean)')
                kde3 = sns.kdeplot(margin_attacked[attcked_edges].cpu().numpy(), ax=ax, label='Att. nodes (pert.)')


                plt.xlim(-1, 1)
                plt.xlabel('Class. margin $\psi$')
                
                if ax.get_ylim()[1] > y_kde_max:
                    ax.set_ylim(0, y_kde_max)
                
                mpl_latex.savefig(
                    f'latex/assets/global_prbcd_kde_nl_{dataset}_{make_undirected}_{epsilon}_{loss_type}_margin_attacked',
                    fig, close_fig=False
                )
                
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                
                plt.gcf().show()
                
                mpl_latex.savefig(
                    f'latex/assets/global_prbcd_kde_l_{dataset}_{make_undirected}_{epsilon}_{loss_type}_margin_attacked',
                    fig, close_fig=False
                )
                
                mpl_latex.dedicated_legend_plot(
                    f'latex/assets/global_prbcd_kde_legend_1col',
                    mod_label=lambda l: l,
                    ncol=1
                )
                mpl_latex.dedicated_legend_plot(
                    f'latex/assets/global_prbcd_kde_legend',
                    mod_label=lambda l: l,
                    ncol=3
                )
                
                ############################################

Training...:   0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 2/400 [00:00<00:22, 17.48it/s]

Clean accuracy: 0.64861661195755
cora_ml False 0.001 CE

Epoch: 0 Loss: 1.0757806301116943 Accuracy: 64.664 %



  6%|▌         | 24/400 [00:01<00:18, 19.82it/s]


Epoch: 20 Loss: 1.0968645811080933 Accuracy: 64.862 %



...

 95%|█████████▌| 381/400 [09:44<00:29,  1.53s/it]


Epoch: 380 Loss: 0.3156895041465759 Accuracy: 30.270 %



100%|██████████| 400/400 [10:13<00:00,  1.53s/it]
/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale_refactoring/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


Perturbed accuracy: 0.46803081035614014


/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale_refactoring/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]
/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale_refactoring/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]
